In [1]:
#Run from server

gpu4 = "GPU-71b4cdfc-e381-0b98-9b24-4fc06284b496" 
gpu5 = "GPU-99d0769a-9f86-4800-a40e-2320dddcf5d1" 
gpu6 = "GPU-7423cfb5-cff4-ec4d-7e96-ea6e1591d56f"
gpu7 = "GPU-c0a8738f-6dd0-1b78-c38f-4969fd3886a8"

import os
#
os.environ["CUDA_VISIBLE_DEVICES"]= gpu6 + "," + gpu5

In [2]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

['/device:CPU:0', '/device:XLA_CPU:0', '/device:XLA_GPU:0', '/device:XLA_GPU:1', '/device:GPU:0', '/device:GPU:1']


In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]

'GPU-7423cfb5-cff4-ec4d-7e96-ea6e1591d56f,GPU-99d0769a-9f86-4800-a40e-2320dddcf5d1'

In [ ]:
#Run from google colab

!pip install SimpleITK

from google.colab import drive
drive.mount('/content/gdrive')


In [ ]:
%cd /content/gdrive/MyDrive/2 TUM/Project/Internships/notebooks

In [4]:
import os
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import tensorflow as tf
from sklearn.model_selection import train_test_split
import cv2
import random
import h5py

In [5]:
def myshow(img):
    nda = sitk.GetArrayViewFromImage(img)
    plt.imshow(nda)

In [6]:
ImgDir = os.path.join("..","data","MICCAI_BraTS2020_TrainingData")
features_path = list()
labels_path = list()
count = 0
limit = 20

for folder in os.listdir(ImgDir):
    count +=1
    if 'Training' in folder and count<limit:
        new_dir = os.path.join(ImgDir,folder)
        data = os.listdir(new_dir)
        for files in data:
            if 'flair' in files:
                features_path.append(os.path.join(new_dir, files))
            if 'seg' in files:
                labels_path.append(os.path.join(new_dir, files))

print(len(features_path))
print(len(labels_path))

X=features_path
Y=labels_path

19
19


In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.1, random_state=1)

In [8]:
batch_size = 10

x_batch = []
y_batch = []

for i in range(batch_size):

    rand = random.choice(range(len(X)))

    x_path = X[rand]
    y_path = Y[rand]

    x = sitk.GetArrayViewFromImage(sitk.ReadImage(x_path))
    y = sitk.GetArrayViewFromImage(sitk.ReadImage(y_path))

    #x = sitk.GetArrayViewFromImage(sitk.Cast((sitk.ReadImage(x_path)),sitk.sitkUInt16))
    #y = sitk.GetArrayViewFromImage(sitk.Cast((sitk.ReadImage(y_path)),sitk.sitkUInt16))

    x = np.moveaxis(x, [0, 1, 2], [-1, -2, -3])
    y = np.moveaxis(y, [0, 1, 2], [-1, -2, -3])

    x = np.resize(x,(240,240,128,1))
    y = np.resize(y,(240,240,128,1))

    x_batch.append(x)
    y_batch.append(y)

x_batch = np.array(x_batch)
y_batch = np.array(y_batch)

In [9]:
x_batch.shape, y_batch.shape

((10, 240, 240, 128, 1), (10, 240, 240, 128, 1))

In [10]:
flair = x_batch
label_full = y_batch

In [14]:
train_gt.shape

(10, 240, 240, 128, 1)

In [15]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, Conv3D, MaxPooling3D, Conv3DTranspose, AveragePooling3D, ZeroPadding3D
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model

In [16]:
K.set_image_data_format('channels_last')

img_size = 240      #original img size is 240*240
smooth = 0.005 
num_of_aug = 2
num_epoch = 30
pul_seq = 'Flair'
sharp = False       # sharpen filter
LR = 1e-4

num_of_patch = 4 #must be a square number
label_num = 5   

In [17]:
img_depth=128
img_rows=240
img_cols=240

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)
    

def get_unet():
    inputs = Input((img_rows, img_cols,img_depth,1))
    conv1 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling3D(pool_size=(2, 2, 2))(conv1)

    conv2 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling3D(pool_size=(2, 2, 2))(conv2)

    conv3 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling3D(pool_size=(2, 2, 2))(conv3)

    conv4 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling3D(pool_size=(2, 2, 2))(conv4)

    conv5 = Conv3D(512, (3, 3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv3D(512, (3, 3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv3DTranspose(256, (2, 2, 2), strides=(2, 2, 2), padding='same')(conv5), conv4], axis=4)
    conv6 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv3DTranspose(128, (2, 2, 2), strides=(2, 2, 2), padding='same')(conv6), conv3], axis=4)
    conv7 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv3DTranspose(64, (2, 2, 2), strides=(2, 2, 2), padding='same')(conv7), conv2], axis=4)
    conv8 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv3DTranspose(32, (2, 2, 2), strides=(2, 2, 2), padding='same')(conv8), conv1], axis=4)
    conv9 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv3D(1, (1, 1, 1), activation='sigmoid')(conv9)


    model = Model(inputs=[inputs], outputs=[conv10])

    model.summary()
    #plot_model(model, to_file='model.png')

    model.compile(optimizer=Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.000000199), loss=dice_coef_loss, metrics=[dice_coef])

    return model


In [18]:

model2=get_unet()
model2.summary

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 240, 240, 12 0                                            
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, 240, 240, 128 896         input_1[0][0]                    
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 240, 240, 128 27680       conv3d[0][0]                     
__________________________________________________________________________________________________
max_pooling3d (MaxPooling3D)    (None, 120, 120, 64, 0           conv3d_1[0][0]                   
_______________________________________________________________________________________

<bound method Model.summary of <tensorflow.python.keras.engine.functional.Functional object at 0x7f3638106240>>

In [19]:
model2.fit(flair,label_full, batch_size=1, epochs=1, verbose=2, shuffle=True)

TypeError: in user code:

    /home/spablo/virtualenvironment/project_1/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    <ipython-input-17-1c66417cbd30>:13 dice_coef_loss  *
        return 1-dice_coef(y_true, y_pred)
    <ipython-input-17-1c66417cbd30>:8 dice_coef  *
        intersection = K.sum(y_true_f * y_pred_f)
    /home/spablo/virtualenvironment/project_1/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:1140 binary_op_wrapper
        raise e
    /home/spablo/virtualenvironment/project_1/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:1124 binary_op_wrapper
        return func(x, y, name=name)
    /home/spablo/virtualenvironment/project_1/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:1456 _mul_dispatch
        return multiply(x, y, name=name)
    /home/spablo/virtualenvironment/project_1/lib/python3.6/site-packages/tensorflow/python/util/dispatch.